This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5); it also requires the pandas and openpyxl libraries (`sage --pip install openpyxl/pandas`).

In this notebook, we compute cyclic purely geometric extensions with $q=2$ using the upper bound on Weil polynomials. Allow about 2 hours for completion.

In [1]:
load("auxiliary.sage")

Read data about the exhaustion from an Excel spreadsheet. Note that we only use the data about base curves $C$, not covering curves $C'$ (except at the end to match up the results).

In [2]:
import pandas

In [3]:
df = pandas.read_excel('polys.xlsx', index_col=int(0))

In [4]:
if "Unnamed: 0" in df:
    del df["Unnamed: 0"]
if "Cyclic" in df:
    del df["Cyclic"]
print(list(df))

['d', 'g', "g'", 'Label of J(C)', '#J(C)', 'Counts of C', "Counts of C'"]


In [5]:
candidates = {}
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    if (d,g,g1) not in candidates:
        candidates[d,g,g1] = []
    s = r["Label of J(C)"]
    if s not in candidates[d,g,g1]:
        candidates[d,g,g1].append(s)

Table of curves with a given Weil polynomial, taken from LMFDB.

In [6]:
P.<x,y,z,t> = GF(2)[]
curves_by_poly = {}

In [7]:
curves_by_poly[1] = {
    '1.2.ac': [(P(1), x^3+x+1)],
    '1.2.ab': [(x, x^3+x^2+1)],
    '1.2.a': [(P(1), x^3)],
    '1.2.b': [(x, x^3+1)],
    '1.2.c': [(P(1), x^3+x)]
}

In [8]:
curves_by_poly[2] = {
    '2.2.ab_a': [(x,x^5+x^2+x)],
    '2.2.ab_c': [(x, x^5+x^3+x)],
    '2.2.ab_d': [(x^3+x+1, x^5+x^4+1)],
    '2.2.ac_e': [(P(1), x^5+x^4+1)],
    '2.2.a_ab': [(x^2+x, x^5+x^3+x^2+x)],
    '2.2.a_a': [(P(1), x^5)],
    '2.2.a_b': [(x^2+x+1, x^5+x^3+x^2+1)],
    '2.2.a_c': [(P(1), x^5+x^4+x^3)],
    '2.2.b_a': [(x, x^5+x)],
    '2.2.b_b': [(x^3+x+1, x^6+x^3+x^2+x)],
    '2.2.b_c': [(x, x^5+x^3+x^2+x)],
    '2.2.b_d': [(x^3+x+1, x^6+x^5+x^4+x^2)],
    '2.2.c_c': [(P(1), x^5+x^3)],
    '2.2.c_d': [(x^2+x+1, x^5+x^4+x^2+x)], 
    '2.2.c_e': [(P(1),x^5+x^4)],
    '2.2.d_f': [(x^3+x+1,x^5+x^4+x^3+x)],
}

In [9]:
curves_by_poly[3] = {
    '3.2.ad_g_ai': [[(x^4+x^2+1,x^8+x+1)], []],
    '3.2.ac_c_ab': [[(x^4+x^2+x+1, x^8+x^7+x^5+x^3+x^2+1)],
                    [x^4+x^3*y+x^2*z^2+x*z^3+y^4+y^3*z+y^2*z^2]],
    '3.2.ac_e_af': [[(x^4+x^2+x,x^8+x^7+x^5+x^4+x)], []],
    '3.2.ab_a_a': [[(x, x^7+x^6+x), (x^4+x^2,x^8+x^5+x^3+x)],
                   [x^4+x^3*y+x^2*y*z+x*z^3+y^4]],
    '3.2.ab_a_c': [[(x, x^7+x^6+x^5+x^2+x)], 
                   [x^3*y+x^2*y^2+x^2*y*z+x*z^3+y^4,
                    x^3*y+x^2*y^2+x*y^2*z+x*z^3+y^4]],
    '3.2.ab_a_e': [[], [x^4+x^3*y+x^3*z+x^2*y*z+x*y^3+x*y^2*z+x*y*z^2+x*z^3+y^2*z^2]],
    '3.2.ab_c_ac': [[(x, x^7+x^5+x), (x^4+x^2+1, x^5+x+1)],
                    [x^4+x^2*y^2+x^2*y*z+x*z^3+y^4]],
    '3.2.ab_c_ab': [[], [x^4+x^3*y+x^2*y*z+x*z^3+y^4+y^3*z+y^2*z^2]],
    '3.2.ab_c_a': [[(x^4+x^2,x^8+x)], []],
    '3.2.a_a_ab': [[(x^3+x+1, x^7+x^5+x^4+x^3+x+1)], []],
    '3.2.a_a_b': [[(x^3+x+1, x^7+x^6+x^5+x^4+x^3+x^2+x)],
                  [x^4+x^3*z+x^2*y^2+x^2*y*z+x*y*z^2+x*z^3+y^3*z]],
    '3.2.a_a_d': [[], [x^4+x^3*y+x^3*z+x^2*y*z+x*y^3+x*y^2*z+x*z^3+y^4,
                       x^3*z+x^2*y^2+x^2*y*z+x^2*z^2+x*y*z^2+x*z^3+y^3*z]],
    '3.2.a_a_f': [[], [x^3*y+x*z^3+y^3*z]],
    '3.2.a_b_a': [[(x^2+x, x^7+x^5+x^2+x), (x^2+x+1, x^7+x^5+x^3+x^2+x+1)],
                  [x^4+x*y^3+x*z^3+y^4]],
    '3.2.a_b_d': [[], [x^4+x^3*y+x^3*z+x^2*y^2+x*z^3+y^3*z, 
                       x^4+x^3*y+x^3*z+x^2*y^2+x*z^3+y^3*z+y^2*z^2]],
    '3.2.a_c_ab': [[(x^4+x^2+x+1, x^8+x^7+x^4+x^2),
                    (x^4+x^2+x+1, x^6+x^5+x^2+1)],
                   [x^4+x^2*y*z+x*y^3+x*y^2*z+x*z^3+y^4]],
    '3.2.a_c_b': [[(x^4+x^2+x+1, x^7+1), (x^4+x^2+x+1, x^8+x^6+x^5+x^4)],
                  [x^4+x^3*z+x^2*y*z+x*y^3+x*y^2*z+x*z^3+y^2*z^2]],
    '3.2.b_b_b': [[(x^4+x+1, x^8+x^7+x^5+x^3)],
                    [x^3*z+x^2*y*z+x*y^3+x*z^3+y^4]],
    '3.2.b_b_e': [[], [x^3*z+x^2*y^2+x^2*y*z+x*z^3+y^3*z]],
    '3.2.b_c_a': [[(x^4+x^2,x^4+x)], []],
    '3.2.b_c_b': [[], [x^3*y+x^2*y*z+x*z^3+y^3*z]],
    '3.2.b_c_c': [[(x,x^7+x^5+x^2+x), (x^4+x^2+1, x^8+x^5+x^4+x)],
                  [x^4+x^3*y+x^2*y^2+x*y^2*z+x*z^3+y^4]],
    '3.2.b_c_d': [[], [x^4+x^3*y+x^2*y^2+x*y*z^2+x*z^3+y^3*z]],
    '3.2.b_c_e': [[(x, x^7+x), (x^4+x^2, x^2+x), (x^4+x^2+1, x^8+x^4+x^2+x)],
                  [x^4+x^2*y*z+x*z^3+y^4]],
    '3.2.b_d_c': [[], [x^4+x^3*z+x^2*y^2+x^2*y*z+x^2*z^2+x*z^3+y^3*z]],
    '3.2.b_d_e': [[], [x^4+x^3*z+x*y^3+x*y^2*z+x*z^3+y^2*z^2]],
    '3.2.b_e_d': [[], [x^4+x^3*y+x^2*y^2+x^2*y*z+x*z^3+y^3*z]],
    '3.2.b_e_e': [[], [x^4+x^2*y^2+x*y*z^2+x*z^3+y^3*z]],
    '3.2.c_d_d': [[], [x^3*y+x^3*z+x*z^3+y^3*z]],
    '3.2.c_d_f': [[], [x^3*y+x^3*z+x^2*y^2+x^2*z^2+x*z^3+y^3*z]],
    '3.2.c_e_e': [[], [x^3*y+x^3*z+x*z^3+y^4]],
    '3.2.c_e_f': [[(x^4+x^2+x, x^7+x^5+x^2+x)],
                    [x^3*y+x^2*y^2+x^2*z^2+x*z^3+y^3*z]],
    '3.2.c_e_g': [[(P(1), x^7+x^4)], [x^4+x^2*y^2+x*z^3+y^3*z]],
    '3.2.c_e_h': [[(x^4+x^2+x+1, x^5+x^3+x^2+x)], []],
    '3.2.d_g_i': [[(x^4+x^2+1, x^4+x)], []],
    '3.2.d_g_k': [[(x^4+x^2+1, x^5+x^2)], []],
    '3.2.d_h_l': [[(x^4+x+1,x^6+x^5+x^3+x)], []],
}

In [10]:
curves_by_poly[4] = {
    '4.2.a_c_ab_c': [[(x^3+x+1, x^9+x^7+x^2)],
                     [(x*y+t^2, x^3+y^3+x*y*z+z^3+x*y*t+x*z*t),
                      (x*y+t^2, x^3+x^2*y+y^3+x*y*z+z^3+x*y*t+x*z*t)]],
    '4.2.a_c_ab_e': [[(x^3+x+1, x^9+x^6+x^2)], 
                     [(x^2+x*y+y^2+z*t, x^2*z+x*y*z+y^2*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.a_c_ab_g': [[(x^3+x+1, x^9+x^7+x+1)],
                     [(x*y+t^2, y^3+x*y*z+z^3+x^2*t+x*y*t+x*z*t),
                      (x^2+x*y+y^2+z*t, x^3+x^2*y+y^3+y^2*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.a_c_b_c': [[(x^3+x+1, x^9+x^7+x^6+1)],
                    [(x*y+z*t, x^3+y^3+x^2*z+y^2*z+x*z^2+y*z^2+x*t^2),
                     (x*y+z*t, x*y^2+y^3+x^2*z+x*y*z+y^2*z+x*z^2+z^3+x^2*t+x*t^2),
                     (x*y+t^2, x^3+y^3+x*y*z+y^2*z+z^3+x*z*t),
                     (x^2+x*y+y^2+z*t, x^2*y+x*y^2+y*z^2+x^2*t+x*t^2)]],
    '4.2.a_c_b_e': [[(x^3+x+1, x^9+1)],
                    [(x^2+x*y+y^2+z*t, x^3+x^2*y+x*y^2+y^2*z+x*z^2+x^2*t+x*t^2),
                     (x^2+x*y+y^2+z*t, x^2*y+x*y^2+x^2*z+y*z^2+x^2*t+x*t^2)]],
    '4.2.a_c_b_g': [[(x^3+x+1, x^9+x^7+x^6+x^2+x)],
                    [(x^2+x*y+y^2+z*t, y^3+x^2*z+x*z^2+x*t^2)]],
    '4.2.a_c_d_a': [[], [(x^2+x*y+y^2+z*t, x^3+x^2*y+x*y^2+y*z^2+x^2*t+x*t^2)]],
    '4.2.a_c_d_c': [[], [(x^2+x*y+y^2+z*t, x^3+y^3+x^2*z+x*y*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.a_d_ab_f': [[], [(x^2+x*y+y^2+z*t, x^3+x^2*y+x^2*z+y^2*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.a_d_ab_h': [[], [(x*y+t^2, x^3+x^2*y+x*y^2+y^3+x*y*z+z^3+y^2*t+x*z*t)]],
    '4.2.a_d_b_f': [[],
                    [(x*y+z*t, x^3+x*y^2+y^3+x^2*z+x*y*z+y^2*z+x*z^2+y*z^2+x*t^2),
                     (x*y+t^2, x^3+x*y*z+z^3+x^2*t+y^2*t+x*z*t),
                     (x*y+t^2, x^3+y^3+x*y*z+z^3+y^2*t+x*z*t),
                     (x*y+t^2, x^3+x^2*y+y^3+x*y*z+z^3+y^2*t+x*z*t),
                     (x^2+x*y+y^2+z*t, x^3+x^2*y+y^3+x^2*z+x*y*z+x*z^2+x*t^2)]],
    '4.2.a_d_b_h': [[], [(x*y+z*t, x^3+x*y^2+y^3+x^2*z+y^2*z+x*z^2+x^2*t+x*y*t+x*t^2),
                          (x^2+x*y+y^2+z*t, x^3+x^2*y+y^2*z+x*z^2+x*y*t+x*t^2),
                          (x^2+x*y+y^2+z*t, x^3+y^3+x*y*z+y^2*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.a_d_d_f': [[], [(x*y+t^2, x^2*y+y^3+x^2*z+x*y*z+z^3+y^2*t+x*z*t)]],
    '4.2.a_e_b_k': [[], [(x*y+t^2, x^3+x^2*y+y^3+x*y*z+z^3+x^2*t+x*y*t+x*z*t)]],
    '4.2.b_b_a_b': [[], [(x*y+t^2, x^2*y+y^2*z+z^3+x^2*t+y^2*t+x*z*t)]],
    '4.2.b_b_a_d': [[], [(x*y+z*t, x^3+x^2*y+y^3+x^2*z+x*y*z+y^2*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.b_b_c_d': [[], [(x*y+z*t, x*y^2+y^3+x^2*z+x*y*z+x*z^2+x*y*t+x*t^2),
                         (x*y+t^2, x^2*z+y^2*z+z^3+y^2*t+x*z*t)]],
    '4.2.b_b_c_f': [[], [(x*y+z*t, x^2*y+x*y^2+y^3+x^2*z+x*y*z+x*z^2+x*t^2),
                         (x*y+t^2, x^3+x^2*y+x^2*z+y^2*z+z^3+x^2*t+y^2*t+x*z*t)]],
    '4.2.b_b_c_h': [[], [(x*y+t^2, x^3+y^2*z+z^3+y^2*t+x*z*t),
                         (x^2+x*y+y^2+z*t, y^3+x*y*z+x*z^2+x*t^2)]],
    '4.2.b_b_a_b': [[], [(x*y+t^2, x^2*y+y^2*z+z^3+x^2*t+y^2*t+x*z*t)]],
    '4.2.b_b_a_d': [[], [(x*y+z*t, x^3+x^2*y+y^3+x^2*z+x*y*z+y^2*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.b_c_a_a': [[(x^2, x^9+x^7+x+1)], []],
    '4.2.b_c_a_c': [[(x, x^9+x^7+x^5+x^3+x^2+1)], []],
    '4.2.b_c_a_e': [[], [(x^2+x*y+y^2+z*t, x*y*z+y^2*z+x*z^2+x*t^2)]],
    '4.2.b_c_c_c': [[(x, x^9+x^7+x^3+1)], []],
    '4.2.b_c_c_e': [[(x^2, x^9+x^5+x^4+x), (x^2, x^9+x^4+x+1)], []],
    '4.2.b_c_c_g': [[(x, x^9+1)],
                    [(x*y+z*t, x^3+x*y^2+x^2*z+y^2*z+x*z^2+y^2*t+x*t^2),
                     (x*y+z*t, y^3+x^2*z+x*y*z+y^2*z+y*z^2+x^2*t+x*t^2),
                     (x*y+t^2, y^3+x^2*z+x*y*z+z^3+x^2*t),
                     (x^2+x*y+y^2+z*t, x^3+x^2*y+y^3+x^2*z+x*y*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.b_b_c_h': [[], [(x*y+t^2, x^3+y^2*z+z^3+y^2*t+x*z*t),
                         (x^2+x*y+y^2+z*t, y^3+x*y*z+x*z^2+x*t^2)]],
    '4.2.b_c_d_h': [[(x^4+x^2+x, x^9+x^8+x^2+1)], 
                    [(x*y+z*t, x^3+x*y^2+y^3+x*y*z+x*z^2+x*y*t+x*t^2),
                     (x^2+x*y+y^2+z*t, x^2*y+y^3+x*y*z+y^2*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.b_c_d_j': [[], [(x*y+t^2, x^3+x^2*y+z^3+x^2*t+y^2*t+x*z*t)]],
    '4.2.b_c_e_e': [[(x^2, x^9+x^7+x^5+x)], []],
    '4.2.b_d_c_h': [[], [(x*y+z*t, x^3+x^2*y+x*y^2+x*y*z+y^2*z+x*z^2+x^2*t+y^2*t+x*t^2),
                         (x*y+z*t, x^3+y^3+x^2*z+x*y*z+y^2*z+x*z^2+x^2*t+y^2*t+x*t^2),
                         (x*y+t^2, x^3+x^2*y+x*y^2+y^3+z^3+x*y*t+y^2*t+x*z*t)]],
    '4.2.b_d_e_j': [[], [(x*y+t^2, x^3+y^3+z^3+x*y*t+y^2*t+x*z*t)]],
    '4.2.b_e_c_i': [[], [(x*y+t^2, x^3+y^3+z^3+x*z*t)]],
    '4.2.b_e_e_k': [[], [(x*y+t^2, x^2*y+y^3+x^2*z+x*y*z+z^3+x^2*t),
                         (x^2+x*y+y^2+z*t, x^2*y+y^3+x^2*z+x*y*z+y^2*z+x*z^2+x*y*t+x*t^2)]],
    '4.2.c_e_h_k': [[(x^3+x+1, x^9+x^7)], []],
    '4.2.c_g_i_q': [[], 
                    [(x*y+z*t,x^3+x*y^2+x^2*z+y^2*z+x*z^2+y*z^2+x^2*t+x*y*t+y^2*t+x*t^2),
                     (x*y+z*t,y^3+x*y*z+y^2*z+x*z^2+y*z^2+x^2*t+x*y*t+y^2*t+x*t^2)]],
    '4.2.d_f_i_n': [[], [(x*y+z*t, x^2*y+x*y^2+x^2*z+x*y*z+y^2*z+y*z^2+x^2*t+y^2*t+x*t^2)]],
    '4.2.d_f_k_s': [[], 
                    [(x*y+t^2, x^2*y+x^2*z+x*y*z+y^2*z+z^3+x^2*t+x*y*t+y^2*t)]],
    '4.2.d_h_l_r': [[(x^5+x^2+1, x^4+x^2)], 
                    [(x*y+z*t,x^2*y+x*y^2+y^3+x*y*z+y^2*z+x*z^2+x^2*t+y^2*t+x*t^2)]],
    '4.2.d_i_o_x': [[],
                    [(x*y+z*t,x^2*z+x*y*z+x*z^2+y*z^2+x^2*t+y^2*t+x*t^2)]],
    '4.2.e_j_q_z': [[], [(x*y+z*t, x^2*y+y^3+x*y*z+y^2*z+x*z^2+x^2*t+y^2*t+x*t^2)]],
    '4.2.e_k_u_bg': [[], [(x*y+z*t, x^2*z+y^2*z+y*z^2+x^2*t+y^2*t+x*t^2)]],
    '4.2.f_o_bc_bs': [[], [(x*y+z*t, x*y^2+y^3+x^2*z+y^2*z+x*z^2+x^2*t+y^2*t+x*t^2)]]
}

In [11]:
curves_by_poly[5] = {
}

Produce Magma function fields of curves with a given zeta function.

In [12]:
def function_fields_by_weil_poly(s, m=1):
    if m == 1:
        F0 = magma.FunctionField(GF(2))
    else:
        K.<a> = GF(2^m)
        F0 = magma.FunctionField(K)
    F0.AssignNames('x')
    S = magma.PolynomialRing(F0)
    S.AssignNames('y')
    l = s.split(".")
    g = Integer(l[0])
    ans = []
    # Hyperelliptics.
    m = curves_by_poly[g][s] if g <= 2 else curves_by_poly[g][s][0]
    for (u,v) in m:
        yield magma.FunctionField(S.1^2 + u(F0.1, 0, 0, 0)*S.1 + v(F0.1, 0, 0, 0))
    # Plane quartics for g=3.
    if g == 3:
        for u in curves_by_poly[g][s][1]:
            yield magma.FunctionField(u(F0.1, S.1, 1, 0))
    # Genus 4 curves.
    if g == 4:
        for (u,v) in curves_by_poly[g][s][1]:
            if u == x*y + z*t:
                yield magma.FunctionField(v(F0.1, S.1, F0.1*S.1, 1))
            elif u == x*y + t^2:
                yield magma.FunctionField(v(F0.1, S.1^2/F0.1, 1, S.1))
            elif u == x^2 + x*y + y^2 + z*t:
                yield magma.FunctionField(v(F0.1, S.1, 1, F0.1^2 + F0.1*S.1 + S.1^2))
            else:
                raise ValueError

Check that the zeta functions of these curves are as indicated.

In [13]:
P.<T> = QQ[]
for g in range(1, 5):
    for s in curves_by_poly[g]:
        for F in function_fields_by_weil_poly(s):
            u = F.ZetaFunction().Numerator().sage()(T).reverse()
            assert label_from_weil_poly(u) == s

Exhaust over abelian extensions of a Magma function field of a fixed prime degree to find instances of a particular relative Weil polynomial.

In [14]:
def match_weil_poly(F, d, delta=0, verbose=False):
    P.<T> = PolynomialRing(QQ)
    ans = []
    g = F.Genus()
    g1 = g + (d-1)*(g-1) + delta
    for F1 in cyclic_covers(F, d, delta=delta):
        if verbose:
            print(F1.ZetaFunction().Numerator() // F.ZetaFunction().Numerator())
        if F1.ClassNumber() == F.ClassNumber():
            ans.append((F, F1))
    return(ans)

In [15]:
covers_by_poly = {}

In [16]:
def match_weil_poly_general(s, d, delta=0, verbose=False):
    l = s.split(".")
    g = Integer(l[0])
    g1 = g + (d-1)*(g-1) + delta
    ans = []
    for F in function_fields_by_weil_poly(s):
        ans.append((s, match_weil_poly(F, d, delta=delta, verbose=verbose)))
    if ans:
        if (d, g, g1) not in covers_by_poly:
            covers_by_poly[d, g, g1] = {}
        if s not in covers_by_poly[d, g, g1]:
            covers_by_poly[d, g, g1][s] = {}
        for (_, l) in ans:
            for (F, F1) in l:
                if F not in covers_by_poly[d, g, g1][s]:
                    covers_by_poly[d, g, g1][s][F] = []
                if F1 not in covers_by_poly[d, g, g1][s][F]:
                    covers_by_poly[d, g, g1][s][F].append(F1)
    return ans

In [17]:
for (d, g, g1) in candidates:
    print(d, g, g1)
    delta = g1-g-(d-1)*(g-1)
    if g <= 4:
        covers_by_poly[d, g, g1] = {}
        for s in candidates[d,g,g1]:
            match_weil_poly_general(s, d, delta=g1-g-(d-1)*(g-1))

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 3 9
5 2 6
6 2 7
7 2 8


Check that no covers that occur for $d=4, g=2, g'=5$ are dihedral, by comparing options for the intermediate covers with $(d,g,g') = (2,2,3)$ and $(d,g,g') = (2,3,5)$.

In [18]:
covers_by_poly[2,2,3]['2.2.ab_c']

{Algebraic function field defined over Univariate rational function field over GF(2) by
 y^2 + x*y + x^5 + x^3 + x: [Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + x*y + x^5 + x^3 + x by
  $.1^2 + $.1 + x^3 + x + 1]}

In [19]:
covers_by_poly[2,3,5]['3.2.ad_g_ai']

{Algebraic function field defined over Univariate rational function field over GF(2) by
 y^2 + (x^4 + x^2 + 1)*y + x^8 + x + 1: [Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + (x^4 + x^2 + 1)*y + x^8 + x + 1 by
  $.1^2 + $.1 + $.1 + x^2 + 1,
  Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + (x^4 + x^2 + 1)*y + x^8 + x + 1 by
  $.1^2 + $.1 + $.1 + x^2]}

In [20]:
covers_by_poly[4,2,5]['2.2.ab_c']

{Algebraic function field defined over Univariate rational function field over GF(2) by
 y^2 + x*y + x^5 + x^3 + x: [Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + x*y + x^5 + x^3 + x by
  $.1^4 + (x^3 + x)*$.1^2 + (x^3 + x + 1)*$.1 + (x^4 + x)*$.1 + x^10 + x^8 + x^7 + x^3 + x^2 + x + 1,
  Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + x*y + x^5 + x^3 + x by
  $.1^4 + (x^3 + x)*$.1^2 + (x^3 + x + 1)*$.1 + (x^4 + x)*$.1 + x^10 + x^8 + x^7 + x^2 + 1]}

In [21]:
for _, l1 in covers_by_poly[2,3,5]['3.2.ad_g_ai'].items():
    for _, l2 in covers_by_poly[4,2,5]['2.2.ab_c'].items():
        for F1 in l1:
            for F2 in l2:
                print(F1.IsIsomorphic(F2))

true
true
true
true


Identify base curves $C$ when $(d,g,g') = (2,7,13)$ and $\#C(\mathbb{F}_2) = 6$.

In [22]:
P.<T> = QQ[]
u0 = T^6 + 3*T^5 - 3*T^4 - 11*T^3 + 3*T^2 + 9*T - 4
l1 = []
for F in function_fields_by_weil_poly('1.2.a'):
    places3 = [i for i in F.Places(3)]
    z = F.DivisorGroup().Identity()
    M = z + 2*places3[0] + 2*places3[1]
    for F1 in cyclic_covers_by_ramification(F, 2, M, delta=6):
        s = magma_poly_list(F1.ZetaFunction().Numerator() // F.ZetaFunction().Numerator())
        u = P(s).reverse()
        u1 = u.trace_polynomial()[0]
        if u1 == u0:
            l1 += match_weil_poly(F1, 2)

In [23]:
print(l1)

[(Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^3 by
$.1^2 + $.1 + x^2/(x^3 + x^2 + 1), Algebraic function field defined over Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^3 by
$.1^2 + $.1 + x^2/(x^3 + x^2 + 1) by
$.1^2 + $.1 + ((x^6 + x^4 + x^3 + x^2)*$.1 + (x^9 + x^7 + x^4 + x))*w + (x^5 + x^2 + x)*$.1 + x^8 + x^2 + x + 1)]


In [24]:
covers_by_poly[2, 7, 13] = {}
for (F, F1) in l1:
    u = P(magma_poly_list(F.ZetaFunction().Numerator())).reverse()
    s = label_from_weil_poly(u)
    if s not in covers_by_poly[2, 7, 13]:
        covers_by_poly[2, 7, 13][s] = {}
    if F not in covers_by_poly[2, 7, 13][s]:
        covers_by_poly[2, 7, 13][s][F] = []
    if F1 not in covers_by_poly[2, 7, 13][s][F]:
        covers_by_poly[2, 7, 13][s][F].append(F1)

Find all cyclic covers with $(d,g,g') = (2,7,13)$ for which $\#C(\mathbb{F}_2) = 6$.

In [25]:
P.<T> = QQ[]
u0 = weil_poly_from_point_count([6,18,12,18,6,60,174],7).trace_polynomial()[0]
u1 = weil_poly_from_point_count([6,18,12,18,6,72,132],7).trace_polynomial()[0]
u2 = weil_poly_from_point_count([6,18,12,18,6,84,90],7).trace_polynomial()[0]
print(u0)
print(u1)
print(u2)

T^7 + 3*T^6 - 3*T^5 - 11*T^4 + 3*T^3 + 9*T^2 - 4*T
T^7 + 3*T^6 - 3*T^5 - 11*T^4 + 3*T^3 + 9*T^2 - 2*T
T^7 + 3*T^6 - 3*T^5 - 11*T^4 + 3*T^3 + 9*T^2


In [30]:
l1 = []
for s,d in [('1.2.a', 2), ('1.2.b', 3)]:
    for F in function_fields_by_weil_poly(s):
        places3 = [i for i in F.Places(3)]
        places6 = [i for i in F.Places(6)]
        z = F.DivisorGroup().Identity()
        m = [z + 2*i + 2*j for (i,j) in itertools.combinations_with_replacement(places3, 2)] + \
            [z + (4-d)*i for i in places6]
        for M in m:
            print(M)
            for F1 in cyclic_covers_by_ramification(F, d, M, delta=6):
                tmp = magma_poly_list(F1.ZetaFunction().Numerator())
                u = P(tmp).reverse()
                u3 = u.trace_polynomial()[0]
                if (d, u3) in [(2, u0), (2, u1), (3, u2)]:
                    print(u3, F1)
                    l1 += match_weil_poly(F1, 2)

4*(x^3 + x^2 + 1, $.1 + x^2 + x)
2*(x^3 + x^2 + 1, $.1 + x^2 + x) + 2*(x^3 + x^2 + 1, $.1 + x^2 + x + 1)
T^7 + 3*T^6 - 3*T^5 - 11*T^4 + 3*T^3 + 9*T^2 - 4*T Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
y^2 + y + x^3 by
$.1^2 + $.1 + x^2/(x^3 + x^2 + 1)
4*(x^3 + x^2 + 1, $.1 + x^2 + x + 1)
2*(x^3 + x + 1)
2*(x^6 + x^5 + x^4 + x^2 + 1, $.1 + x^5 + x^4 + x^2)
2*(x^6 + x^5 + x^4 + x^2 + 1, $.1 + x^5 + x^4 + x^2 + 1)
2*(x^6 + x + 1, $.1 + x^4 + x^2 + 1)
2*(x^6 + x + 1, $.1 + x^4 + x^2)
2*(x^6 + x^5 + x^4 + x + 1, $.1 + x^5 + x^4 + x^3 + x^2 + 1)
2*(x^6 + x^5 + x^4 + x + 1, $.1 + x^5 + x^4 + x^3 + x^2)
2*(x^6 + x^4 + x^2 + x + 1, $.1 + x^5 + x^4 + x^3)
2*(x^6 + x^4 + x^2 + x + 1, $.1 + x^5 + x^4 + x^3 + 1)
2*(x^6 + x^5 + x^3 + x^2 + 1, $.1 + x^5 + x^4 + x^3 + x^2 + x)
2*(x^6 + x^5 + x^3 + x^2 + 1, $.1 + x^5 + x^4 + x^3 + x^2 + x + 1)
(x^3 + x + 1)
(x^3 + x^2 + 1)
(x^6 + x^3 + 1, $.1 + x^5 + x^3 + x^2 + x + 1)
(x^6

In [31]:
covers_by_poly[2, 7, 13] = {}
for (F, F1) in l1:
    u = P(magma_poly_list(F.ZetaFunction().Numerator())).reverse()
    s = label_from_weil_poly(u)
    if s not in covers_by_poly[2, 7, 13]:
        covers_by_poly[2, 7, 13][s] = {}
    if F not in covers_by_poly[2, 7, 13][s]:
        covers_by_poly[2, 7, 13][s][F] = []
    if F1 not in covers_by_poly[2, 7, 13][s][F]:
        covers_by_poly[2, 7, 13][s][F].append(F1)

In [32]:
covers_by_poly[2, 7, 13]

{'7.2.d_l_z_cf_dx_gs_js': {Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
  y^2 + y + x^3 by
  $.1^2 + $.1 + x^2/(x^3 + x^2 + 1): [Algebraic function field defined over Algebraic function field defined over Algebraic function field defined over Univariate rational function field over GF(2) by
   y^2 + y + x^3 by
   $.1^2 + $.1 + x^2/(x^3 + x^2 + 1) by
   $.1^2 + $.1 + ((x^6 + x^4 + x^3 + x^2)*$.1 + (x^9 + x^7 + x^4 + x))*w + (x^5 + x^2 + x)*$.1 + x^8 + x^2 + x + 1]}}

Convert the covers into numerical data in preparation for writing out to the spreadsheet.

In [33]:
l = []
for (d, g, g1) in candidates:
    print(d, g, g1)
    if (d, g, g1) in covers_by_poly:
        for s in covers_by_poly[d, g, g1]:
            for F in covers_by_poly[d, g, g1][s]:
                for F1 in covers_by_poly[d, g, g1][s][F]:
                    u1 = T.parent()(magma_poly_list(F.ZetaFunction().Numerator())).reverse()
                    v1 = T.parent()(magma_poly_list(F1.ZetaFunction().Numerator())).reverse()
                    l.append((d, g, g1, point_count_from_weil_poly(u1, 13),
                              point_count_from_weil_poly(v1, 13)))

2 2 3
2 2 4
2 2 5
2 3 5
2 3 6
2 4 7
2 4 8
2 5 9
2 6 11
2 7 13
3 2 4
3 2 6
3 3 7
3 4 10
4 2 5
4 3 9
5 2 6
6 2 7
7 2 8


Write the results back to the spreadsheet. In the process, we check that every covering we found gives a pair of Weil polynomials from our original list.

In [34]:
df['Cyclic'] = 'No'
used_pols = []
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    ct1 = eval(r["Counts of C"])
    ct2 = eval(r["Counts of C'"])
    if (d, g, g1, ct1, ct2) in l:
        df.loc[i, "Cyclic"] = "Yes"
        used_pols.append((d, g, g1, ct1, ct2))
    elif d == 2 and (g == 5 or g == 6 or (g == 7 and ct1[0] == 7)):
        df.loc[i, "Cyclic"] = "Unknown"
assert all(t in used_pols for t in l)

In [35]:
df.to_excel('polys.xlsx', sheet_name='Weil polynomials', merge_cells=True, freeze_panes=(int(1),int(1)))